# Manipulate maps (.nii files)
The maps of activation for the HMM states are .nii files. This notebook manipulates these files to create new maps (subtract gloabal average from state activity).

In [1]:
from pathlib import Path

import nibabel as nib
from nibabel import cifti2 as ci
import numpy as np
import scipy


# Setup data path
data_path = Path('C:/Users/tobia/Documents/dev/Thesis/thesis/Data')

hmm_states = 12

### Create new maps with GA subtracted

In [2]:
def subtract_global_average(path):
    """Keep the last state as global average and subtract it from all other states."""
    nii = nib.load(path)
    
    data = nii.get_fdata()

    data_stds = np.zeros(data.shape)

    for state in range(hmm_states):
        data_stds[state] = data[state] - data[-1]
        
    # Create a new image object with the result data and header from the first image
    data_nii_stds = ci.Cifti2Image(data_stds, nii.header)
    
    return data_nii_stds

In [15]:
hemispheres = ['left', 'right']
band_types =  {
    'fac1': 'deltatheta',
    'fac2': 'alpha',
    'fac3': 'beta',
    'wideband': 'wideband'
}

for k, v in band_types.items():
    for hemisphere in hemispheres:
        path = Path(f"{data_path}/maps/MRC-Notts_3pcc_embedded_K12_rep1_stdised_{k}_2mm_{hemisphere}.dtseries.nii")
        data_norm = subtract_global_average(path)
        nib.save(data_norm, Path(f"{data_path}/maps_normalized/{hemisphere}_{v}_norm.dtseries.nii"))

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


### Prepare maps for conversion to .nii
This is required for converting to neurosynth.

In [16]:
# Convert to func.gii file format
hemispheres = ['left', 'right']
band_types = ['deltatheta', 'alpha', 'beta', 'wideband']

for band_type in band_types:
    for hemisphere in hemispheres:
        nii = nib.load(Path(f"{data_path}/maps_normalized/{hemisphere}_{band_type}_norm.dtseries.nii"))
        data = nii.get_fdata()
        
        for state in range(hmm_states):
            state_data = data[state]
            state_data = state_data.astype(np.float32)
            meta = nib.gifti.GiftiMetaData.from_dict({"AnatomicalStructurePrimary": f"Cortex{hemisphere.capitalize()}"})
            gii_img = nib.gifti.GiftiImage(darrays=[nib.gifti.GiftiDataArray(state_data)], meta=meta)
            nib.save(gii_img, f'data/maps_normalized_volumetric/temp/{state}_{hemisphere}_{band_type}.func.gii')

C:\Users\tobia\AppData\Local\Temp\ipykernel_21632\818043617.py:12: DeprecationWarning: from_dict class method deprecated. Use GiftiMetaData directly.

* deprecated from version: 4.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 6.0
  meta = nib.gifti.GiftiMetaData.from_dict({"AnatomicalStructurePrimary": f"Cortex{hemisphere.capitalize()}"})


Afterwards, map to the HCP S1200 group average using `wb_command`:  
> ```wb_command -metric-to-volume-mapping maps_conversion/cortical_state_maps/z_state3_left_fac3.func.gii HCP_S1200_GroupAvg/S1200.L.midthickness_MSMAll.32k_fs_LR.surf.gii HCP_S1200_GroupAvg/S1200_AverageT1w_restore.nii maps_conversion/state3_left_fac3.nii -ribbon-constrained HCP_S1200_GroupAvg/S1200.L.white_MSMAll.32k_fs_LR.surf.gii HCP_S1200_GroupAvg/S1200.L.pial_MSMAll.32k_fs_LR.surf.gii```

In [2]:
# Run workbench command for mapping to volumetric space
import subprocess

for hemisphere in ['left', 'right']:
    for fac in ['deltatheta', 'alpha', 'beta', 'wideband']:
        for state in range(hmm_states):
            wb_command = f"wb_command -metric-to-volume-mapping maps_normalized_volumetric/temp/{state}_{hemisphere}_{fac}.func.gii HCP_S1200_GroupAvg/S1200.{'L' if hemisphere == 'left' else 'R'}.midthickness_MSMAll.32k_fs_LR.surf.gii HCP_S1200_GroupAvg/S1200_AverageT1w_restore.nii maps_normalized_volumetric/leftright/{state}_{hemisphere}_{fac}.nii -ribbon-constrained HCP_S1200_GroupAvg/S1200.{'L' if hemisphere == 'left' else 'R'}.white_MSMAll.32k_fs_LR.surf.gii HCP_S1200_GroupAvg/S1200.{'L' if hemisphere == 'left' else 'R'}.pial_MSMAll.32k_fs_LR.surf.gii"
            cd = "C:\\Users\\tobia\\Documents\\dev\\Thesis\\thesis\\data"

            command_string = f'cd {cd} && {wb_command}'

            result = subprocess.run(command_string, shell=True, capture_output=True, text=True)

In [ ]:
# Combine the volumetric maps
for state in range(hmm_states):
    for fac in ['deltatheta', 'alpha', 'beta', 'wideband']:
        # Patch together images
        left_img = nib.load(f'data/maps_normalized_volumetric/leftright/{state}_left_{fac}.nii')
        right_img = nib.load(f'data/maps_normalized_volumetric/leftright/{state}_right_{fac}.nii')

        left_data = left_img.get_fdata()
        right_data = right_img.get_fdata()

        combined_data = left_data + right_data

        combined_img = nib.Nifti1Image(combined_data, left_img.affine, left_img.header)

        combined_img.to_filename(f'data/maps_normalized_volumetric/state{state}_{fac}.nii')

### Maps comparison

In [26]:
s1 = 4
s2 = 3
for hemisphere in ['left', 'right']:
    for band_type in ['fac1', 'fac2', 'fac3', 'wideband']:
        path = Path(f"{data_path}/maps/MRC-Notts_3pcc_embedded_K12_rep1_stdised_{band_type}_2mm_{hemisphere}.dtseries.nii")

        nii = nib.load(path)

        data = nii.get_fdata()

        data_stds = np.zeros(data.shape)

        for state in range(hmm_states):
            data_stds[state] = data[state] - data[-1]
            
        data_s4_minus_s11 = np.zeros((data.shape))

        data_s4_minus_s11[0] = data_stds[s1] - data_stds[s2]

        data_s11_nii_stds = ci.Cifti2Image(data_s4_minus_s11, nii.header)

        nib.save(data_s11_nii_stds, Path(f"temp/{hemisphere}_{band_type}_{s1}-{s2}.dtseries.nii"))

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
